In [3]:
classpath.addInConfig("config")(
"org.nd4j"%"nd4j-native-platform"%"0.9.1"
)

: 

In [4]:
classpath.repositories

res3: Seq[String] = List(
  "ivy2Local",
  "https://repo1.maven.org/maven2",
  "https://oss.sonatype.org/content/repositories/releases",
  "https://repo1.maven.org"
)